<a href="https://colab.research.google.com/github/afairley19/nlp_colab/blob/master/fake_news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"] 
! java -version
# Install pyspark
! pip install --ignore-installed pyspark==2.4.4
# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.6.3

openjdk version "1.8.0_282"
OpenJDK Runtime Environment (build 1.8.0_282-8u282-b08-0ubuntu1~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.282-b08, mixed mode)
Processing /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471/pyspark-2.4.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e3/53/c737818eb9a7dc32a7cd4f1396e787bd94200c3997c72c1dbe028587bd76/py4j-0.10.7-py2.py3-none-any.whl


  Using cached https://files.pythonhosted.org/packages/84/84/3f15673db521fbc4e8e0ec3677a019ba1458b2cb70f0f7738c221511ef32/spark_nlp-2.6.3-py2.py3-none-any.whl


In [1]:
! sudo update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
  0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
* 2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 


In [2]:
from pyspark.sql import SparkSession 
from pyspark.ml import Pipeline
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline 
import sparknlp
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version()) 
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.6.3
Apache Spark version:  2.4.4


In [20]:
from google.colab import files
import io

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
data_path = 'fake_or_real_news.csv'
with open(data_path, 'r') as f:
    lines = f.read().split('\n')

Saving fake_or_real_news.csv to fake_or_real_news (2).csv
User uploaded file "fake_or_real_news.csv" with length 30696129 bytes


In [59]:
file_location = r'./fake_or_real_news.csv'
file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","
df = spark.read.format(file_type)\
.option("inferSchema", infer_schema)\
.option("header", first_row_is_header)\
.option("escape", " ")\
.option("quote", "")\
.load(file_location)


df.printSchema()
df.count()


root
 |-- _c0: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)



104389

In [60]:
df2 = df.na.drop()
df2.printSchema()
df2.count()

root
 |-- _c0: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)



40778

In [49]:
# Preprocessing (Pipeline)
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

use = UniversalSentenceEncoder.pretrained(lang="en")\
.setInputCols(["document"])\
.setOutputCol("sentence_embeddings")

model_name = 'classifierdl_use_fakenews' # pre-trained model name 
document_classifier = ClassifierDLModel.pretrained(model_name)\
.setInputCols(['document', 'sentence_embeddings']).setOutputCol("prediction")

nlp_Pipeline = Pipeline( stages=[documentAssembler,
            use,
            document_classifier])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[OK!]


In [61]:
# Training (Pipeline)
empty_df = spark.createDataFrame([['']]).toDF("text") 
pipelineModel = nlp_Pipeline.fit(empty_df)
result_df = pipelineModel.transform(df2)

# Show Schema
result_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- met

In [62]:
# Visualize the results
import pyspark.sql.functions as F 
result_df.select(F.explode(F.arrays_zip('prediction.result', 'document.result')).alias("cols"))\
.select(F.expr("cols['0']").alias("prediction"), F.expr("cols['1']").alias("document")).show(truncate=False)

+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction|document                                                                                                                                                                                                                            |
+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|FAKE      |"Daniel Greenfield                                                                                                                                                                                                                  |
|REAL      | Fairly or Not” prac